In [0]:
df_appearances = spark.read.csv("/FileStore/tables/appearances.csv", header=True, inferSchema=True)
df_clubs = spark.read.csv("/FileStore/tables/clubs.csv", header=True, inferSchema=True)
df_transfers = spark.read.csv("/FileStore/tables/transfers.csv", header=True, inferSchema=True)
df_games = spark.read.csv("/FileStore/tables/games.csv", header=True, inferSchema=True)
df_players = spark.read.csv("/FileStore/tables/players.csv", header=True, inferSchema=True)


In [0]:
from pyspark.sql import functions as F

# Danh sách các bảng cần cập nhật cột ngày
tables = ["appearances", "games",]

for table in tables:
    df = spark.read.csv(f"/FileStore/tables/{table}.csv", header=True, inferSchema=True)
    
    # Chuyển đổi số nguyên thành ngày (epoch-based)
    df = df.withColumn("date", F.expr("date_add('1970-01-01', CAST(date AS INT))"))
    
    # Lưu lại dữ liệu sau khi xử lý
    df.write.mode("overwrite").parquet(f"dbfs:/tmp/{table}.parquet")

# Xử lý bảng transfers riêng (có cột transfer_date)
df_transfers = spark.read.csv("/FileStore/tables/transfers.csv", header=True, inferSchema=True)
df_transfers = df_transfers.withColumn("transfer_date", F.expr("date_add('1970-01-01', CAST(transfer_date AS INT))"))
df_transfers.write.mode("overwrite").parquet("dbfs:/tmp/transfers.parquet")


In [0]:
from pyspark.sql import functions as F

# Đọc dữ liệu từ DBFS
df_appearances = spark.read.csv("/FileStore/tables/appearances.csv", header=True, inferSchema=True)
df_games = spark.read.csv("/FileStore/tables/games.csv", header=True, inferSchema=True)

# Đổi tên cột 'competition_id' trong df_games để tránh xung đột
df_games = df_games.withColumnRenamed("competition_id", "competition_id_game")
df_games = df_games.withColumnRenamed("date", "game_date")

# Thực hiện join hai bảng dựa trên game_id
df_appearances2 = df_appearances.join(df_games, on="game_id", how="inner") \
    .withColumn("away_home", 
                F.when(F.col("player_club_id") == F.col("home_club_id"), "home")
                 .otherwise("away")) \
    .withColumn("result", 
                F.when((F.col("player_club_id") == F.col("home_club_id")) & (F.col("home_club_goals") > F.col("away_club_goals")), "W")
                 .when((F.col("player_club_id") == F.col("home_club_id")) & (F.col("home_club_goals") == F.col("away_club_goals")), "D")
                 .when((F.col("player_club_id") == F.col("home_club_id")) & (F.col("home_club_goals") < F.col("away_club_goals")), "L")
                 .when((F.col("player_club_id") == F.col("away_club_id")) & (F.col("away_club_goals") > F.col("home_club_goals")), "W")
                 .when((F.col("player_club_id") == F.col("away_club_id")) & (F.col("away_club_goals") == F.col("home_club_goals")), "D")
                 .otherwise("L")) \
    .withColumn("score", F.concat_ws("-", F.col("home_club_goals"), F.col("away_club_goals"))) \
    .withColumn("GF", 
                F.when(F.col("player_club_id") == F.col("home_club_id"), F.col("home_club_goals"))
                 .otherwise(F.col("away_club_goals"))) \
    .withColumn("GA", 
                F.when(F.col("player_club_id") == F.col("home_club_id"), F.col("away_club_goals"))
                 .otherwise(F.col("home_club_goals"))) \
    .withColumn("manager_name", 
                F.when(F.col("player_club_id") == F.col("home_club_id"), F.col("home_club_manager_name"))
                 .otherwise(F.col("away_club_manager_name"))) \
    .withColumn("attendance", F.col("attendance"))

# Lưu bảng appearances2 dưới dạng Parquet
df_appearances2.write.mode("overwrite").parquet("dbfs:/tmp/appearances2.parquet")


In [0]:
from pyspark.sql import functions as F

# Read in the games and clubs data
df_games = spark.read.csv("/FileStore/tables/games.csv", header=True, inferSchema=True)
df_clubs = spark.read.parquet("dbfs:/tmp/clubs.parquet")

# Calculate home game stats
home_games = df_games.filter(F.col("home_club_name").isNotNull()).groupBy(
    "season", "home_club_id", "home_club_name"
).agg(
    F.count("game_id").alias("total_games_home"),
    F.sum(F.when(F.col("home_club_goals") > F.col("away_club_goals"), 1).otherwise(0)).alias("win_home"),
    F.sum(F.when(F.col("home_club_goals") == F.col("away_club_goals"), 1).otherwise(0)).alias("draw_home"),
    F.sum(F.when(F.col("home_club_goals") < F.col("away_club_goals"), 1).otherwise(0)).alias("lost_home"),
    F.sum("home_club_goals").alias("GF_home"),
    F.sum("away_club_goals").alias("GA_home")
)

# Calculate away game stats
away_games = df_games.filter(F.col("away_club_name").isNotNull()).groupBy(
    "season", "away_club_id", "away_club_name"
).agg(
    F.count("game_id").alias("total_games_away"),
    F.sum(F.when(F.col("away_club_goals") > F.col("home_club_goals"), 1).otherwise(0)).alias("win_away"),
    F.sum(F.when(F.col("away_club_goals") == F.col("home_club_goals"), 1).otherwise(0)).alias("draw_away"),
    F.sum(F.when(F.col("away_club_goals") < F.col("home_club_goals"), 1).otherwise(0)).alias("lost_away"),
    F.sum("away_club_goals").alias("GF_away"),
    F.sum("home_club_goals").alias("GA_away")
)

home_games = home_games.alias("home")
away_games = away_games.alias("away")

# Join home and away stats on season and club_id
team_stats = home_games.join(
    away_games, 
    (F.col("home.season") == F.col("away.season")) & (F.col("home.home_club_id") == F.col("away.away_club_id")),
    how="outer"
).select(
    F.coalesce(F.col("home.season"), F.col("away.season")).alias("season"),
    F.coalesce(F.col("home.home_club_id"), F.col("away.away_club_id")).alias("club_id"),
    F.coalesce(F.col("home.home_club_name"), F.col("away.away_club_name")).alias("club_name"),
    
    (F.coalesce(F.col("home.total_games_home"), F.lit(0)) + F.coalesce(F.col("away.total_games_away"), F.lit(0))).alias("total_games"),
    (F.coalesce(F.col("home.win_home"), F.lit(0)) + F.coalesce(F.col("away.win_away"), F.lit(0))).alias("win"),
    (F.coalesce(F.col("home.draw_home"), F.lit(0)) + F.coalesce(F.col("away.draw_away"), F.lit(0))).alias("draw"),
    (F.coalesce(F.col("home.lost_home"), F.lit(0)) + F.coalesce(F.col("away.lost_away"), F.lit(0))).alias("lost"),

    # Corrected performance formula
    F.round(
        (3.0 * (F.coalesce(F.col("home.win_home"), F.lit(0)) + F.coalesce(F.col("away.win_away"), F.lit(0))) +
         (F.coalesce(F.col("home.draw_home"), F.lit(0)) + F.coalesce(F.col("away.draw_away"), F.lit(0)))) /
        (F.coalesce(F.col("home.total_games_home"), F.lit(0)) + F.coalesce(F.col("away.total_games_away"), F.lit(0))) * 100 /3, 2
    ).alias("performance"),

    # Corrected GF, GA, GD calculations
    (F.coalesce(F.col("home.GF_home"), F.lit(0)) + F.coalesce(F.col("away.GF_away"), F.lit(0))).alias("GF"),
    (F.coalesce(F.col("home.GA_home"), F.lit(0)) + F.coalesce(F.col("away.GA_away"), F.lit(0))).alias("GA"),
    F.round(
        (F.coalesce(F.col("home.GF_home"), F.lit(0)) + F.coalesce(F.col("away.GF_away"), F.lit(0))) -
        (F.coalesce(F.col("home.GA_home"), F.lit(0)) + F.coalesce(F.col("away.GA_away"), F.lit(0))), 2
    ).alias("GD")
)



# Add club info
club_info = df_clubs.select("club_id", "domestic_competition_id")

team_stats = team_stats.join(
    club_info, team_stats["club_id"] == club_info["club_id"], how="left"
).drop(club_info["club_id"])


# Save to Parquet or insert into database
team_stats.write.mode("overwrite").parquet("dbfs:/tmp/team_stats.parquet")


In [0]:
# Import necessary libraries
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Read the transfer data from CSV
df_transfers = spark.read.csv("/FileStore/tables/transfers.csv", header=True, inferSchema=True)

# Clean the data: Convert 'transfer_fee' and 'market_value_in_eur' columns to numeric values
df_transfers = df_transfers.withColumn(
    "transfer_fee", F.coalesce(F.col("transfer_fee").cast("double"), F.lit(0))
).withColumn(
    "market_value_in_eur", F.coalesce(F.col("market_value_in_eur").cast("double"), F.lit(0))
)

# Grouping data for outgoing transfers (from_club_id)
transfer_summary_out = df_transfers.groupBy("transfer_season", "from_club_id").agg(
    F.sum(F.when(F.col("transfer_fee").isNotNull(), F.col("transfer_fee")).otherwise(0)).alias("total_transfer_fee_out"),
    F.sum(F.when(F.col("market_value_in_eur").isNotNull(), F.col("market_value_in_eur")).otherwise(0)).alias("total_market_value_out")
)

# Grouping data for incoming transfers (to_club_id)
transfer_summary_in = df_transfers.groupBy("transfer_season", "to_club_id").agg(
    F.sum(F.when(F.col("transfer_fee").isNotNull(), F.col("transfer_fee")).otherwise(0)).alias("total_transfer_fee_in"),
    F.sum(F.when(F.col("market_value_in_eur").isNotNull(), F.col("market_value_in_eur")).otherwise(0)).alias("total_market_value_in")
)

# Renaming columns for consistency
transfer_summary_out = transfer_summary_out.withColumnRenamed("from_club_id", "club_id")
transfer_summary_in = transfer_summary_in.withColumnRenamed("to_club_id", "club_id")

# Merging the two summaries (outgoing and incoming transfers)
transfer_summary = transfer_summary_out.join(
    transfer_summary_in,
    on=["transfer_season", "club_id"],
    how="outer"
).fillna(0)

# Calculating total transfer fee (subtracting outgoing transfer fees and adding incoming)
transfer_summary = transfer_summary.withColumn(
    "total_transfer_fee", F.col("total_transfer_fee_in") - F.col("total_transfer_fee_out")
).withColumn(
    "total_market_value", F.col("total_market_value_in") - F.col("total_market_value_out")
)

# Dropping intermediate columns (total_transfer_fee_out, total_transfer_fee_in, etc.)
transfer_summary = transfer_summary.drop("total_transfer_fee_out", "total_transfer_fee_in", "total_market_value_out", "total_market_value_in")

# Convert season format (extract first two digits and convert to integer)
transfer_summary = transfer_summary.withColumn(
    "season", F.col("transfer_season").substr(1, 2).cast("int") + 2000
)

# Load the team stats data (assuming it's already loaded as df_team_stats)
df_team_stats = spark.read.parquet("dbfs:/tmp/team_stats.parquet")  # Replace with actual path

# Merge the team stats with the transfer summary
df_team_stats = df_team_stats.join(
    transfer_summary,
    on=["season", "club_id"],
    how="left"
)

# Fill any NaN values with 0
df_team_stats = df_team_stats.fillna(0)

# Sort data by club_id and season
df_team_stats = df_team_stats.orderBy("club_id", "season")

# Cumulative sum for transfer_fee and market_value per club
df_team_stats = df_team_stats.withColumn(
    "total_transfer_fee", F.sum("total_transfer_fee").over(Window.partitionBy("club_id").orderBy("season"))
).withColumn(
    "total_market_value", F.sum("total_market_value").over(Window.partitionBy("club_id").orderBy("season"))
)

# Show the final result

df_team_stats.write.mode("overwrite").parquet("dbfs:/tmp/team_stats_with_transfers.parquet")

In [0]:
df_appearances2 = spark.read.parquet("dbfs:/tmp/appearances2.parquet")
df_clubs = spark.read.parquet("dbfs:/tmp/clubs.parquet")
from pyspark.sql import functions as F

# Define Big Five leagues and seasons
big_five_leagues = ['L1', 'GB1', 'ES1', 'FR1', 'IT1']
start_date = '2014-07-01'
end_date = '2024-06-30'

# Filter data for the required leagues and seasons
# Perform the join with clubs to get domestic_competition_id
df_filtered = df_appearances2.join(
    df_clubs, df_appearances2["player_club_id"] == df_clubs["club_id"], how="inner"
).filter(
    (F.col("date") >= start_date) &
    (F.col("date") <= end_date) &
    (F.col("domestic_competition_id").isin(big_five_leagues))
)

df_player_data = df_filtered.groupBy("player_id").agg(
    F.count("*").alias("games_played"),
    F.sum("yellow_cards").alias("total_yellow_cards"),
    F.sum("red_cards").alias("total_red_cards"),
    F.sum("goals").alias("total_goals"),
    F.sum("assists").alias("total_assists"),
    F.sum("GF").alias("total_goals_for_team"),
    F.sum("GA").alias("total_goals_against_team"),
    (F.round(F.avg("yellow_cards"), 2)).alias("average_yellow_cards"),
    (F.round(F.avg("red_cards"), 2)).alias("average_red_cards"),
    (F.round(F.avg("goals"), 2)).alias("average_goals"),
    (F.round(F.avg("assists"), 2)).alias("average_assists"),
    (F.round(F.avg("GF"), 2)).alias("average_goals_for_team"),
    (F.round(F.avg("GA"), 2)).alias("average_goals_against_team"),
    (F.round(F.avg("minutes_played"), 2)).alias("avg_minutes_played"),
    F.sum(F.when(F.col("result") == 'W', 1).otherwise(0)).alias("total_wins"),
    F.sum(F.when(F.col("result") == 'D', 1).otherwise(0)).alias("total_draws"),
    F.sum(F.when(F.col("result") == 'L', 1).otherwise(0)).alias("total_losses"),
    (F.round(
        (3.0 * F.sum(F.when(F.col("result") == 'W', 1).otherwise(0)) + 
         1.0 * F.sum(F.when(F.col("result") == 'D', 1).otherwise(0))) / 
        (3.0 * F.count("*")) * 100, 2)
    ).alias("performance")
)

# Filter players who have played at least 100 games
df_player_data = df_player_data.filter(F.col("games_played") >= 100)
df_players = spark.read.csv("/FileStore/tables/players.csv", header=True, inferSchema=True)

df_top_players = df_player_data.join(
    df_players, df_player_data["player_id"] == df_players["player_id"], how="left"
).select(
    df_player_data["player_id"],
    df_players["name"].alias("player_name"),
    df_player_data["games_played"],
    df_player_data["performance"],
    df_player_data["total_goals"],
    df_player_data["total_assists"],
    df_player_data["total_yellow_cards"],
    df_player_data["total_red_cards"],
    df_player_data["total_goals_for_team"],
    df_player_data["total_goals_against_team"],
    df_player_data["average_yellow_cards"],
    df_player_data["average_red_cards"],
    df_player_data["average_goals"],
    df_player_data["average_assists"],
    df_player_data["average_goals_for_team"],
    df_player_data["average_goals_against_team"],
    df_player_data["avg_minutes_played"],
    df_player_data["total_wins"],
    df_player_data["total_draws"],
    df_player_data["total_losses"]
)

# Sort by performance in descending order and get top 10 players
df_top_players_10 = df_top_players.orderBy(F.col("performance"), ascending=False).limit(10)
df_top_score_10 = df_top_players.orderBy(F.col("total_goals"), ascending=False).limit(10)

df_top_players_10.write.format("delta").mode("overwrite").saveAsTable("my_database.top_players")
df_top_score_10.write.format("delta").mode("overwrite").saveAsTable("my_database.top_scorers")

# Show top players


In [0]:
from pyspark.sql import functions as F

# Define Big Five leagues and seasons
big_five_leagues = ['L1', 'GB1', 'ES1', 'FR1', 'IT1']
seasons = list(range(2014, 2024))

team_stats_with_transfers = spark.read.parquet("dbfs:/tmp/team_stats_with_transfers.parquet")
# Filter data for the required leagues and seasons
df_top_teams = team_stats_with_transfers.filter(
    (F.col("domestic_competition_id").isin(big_five_leagues)) &
    (F.col("season").isin(seasons)) &
    (F.col("total_games") > 10)
)

# Select and sort by performance in descending order
df_top_teams = df_top_teams.select(
    "season",
    "club_id",
    "club_name",
    F.col("domestic_competition_id").alias("dom_league"),
    "performance",
    "total_games",
    "win",
    "draw",
    "lost",
    "GF",
    "GA",
    "GD"
).orderBy(F.col("performance").desc())

# Get the top 10 teams
df_top_teams_by_performance_and_season = df_top_teams.limit(10)
df_top_teams_by_performance_and_season.write.format("delta").mode("overwrite").saveAsTable("my_database.top_teams_by_performance_and_season")
# Show the result
df_top_teams.show()


In [0]:
team_stats  = spark.read.parquet("dbfs:/tmp/team_stats.parquet")
from pyspark.sql import functions as F

# Định nghĩa danh sách các giải đấu lớn
big_five_leagues = ['L1', 'GB1', 'ES1', 'FR1', 'IT1']

# Lọc dữ liệu theo giải đấu và số trận tối thiểu
df_top_teams = team_stats.filter(
    (F.col("domestic_competition_id").isin(big_five_leagues)) &
    (F.col("season").between(2014, 2023)) &  # Giới hạn mùa giải từ 2014 đến 2023
    (F.col("total_games") > 10)  # Chỉ xét đội có hơn 10 trận đấu
)

# Tính tổng số trận, thắng, hòa, thua, số bàn thắng, số bàn thua
df_top_teams = df_top_teams.groupBy(
    "club_id", "club_name", "domestic_competition_id"
).agg(
    F.sum("total_games").alias("total_games"),
    F.sum("win").alias("win"),
    F.sum("draw").alias("draw"),
    F.sum("lost").alias("lost"),
    F.sum("GF").alias("GF"),
    F.sum("GA").alias("GA"),
    (F.sum("GF") - F.sum("GA")).alias("GD"),
    # Tính hiệu suất (performance) = ((Tổng điểm đạt được) / (Tổng điểm tối đa)) * 100
    F.round(
        ((F.sum("win") * 3.0 + F.sum("draw")) * 100.0) / (F.sum("total_games") * 3.0), 2
    ).alias("performance")
)

# Sắp xếp theo hiệu suất giảm dần và lấy top 10
df_top_teams_by_performance_last_decade = df_top_teams.orderBy(F.col("performance").desc()).limit(10)
df_top_teams_by_performance_last_decade.write.format("delta").mode("overwrite").saveAsTable("my_database.top_teams_by_performance_last_decade")
# Hiển thị kết quả
# df_top_teams.show()


In [0]:
from pyspark.sql import functions as F

# "total_transfer_fee", "total_transfer_fee"
team_stats_with_transfers = spark.read.parquet("dbfs:/tmp/team_stats_with_transfers.parquet")
# Define Big Five leagues and seasons
big_five_leagues = ['L1', 'GB1', 'ES1', 'FR1', 'IT1']
seasons = list(range(2014, 2024))

# Filter data for the required leagues and seasons
df_top_teams = team_stats_with_transfers.filter(
    (F.col("domestic_competition_id").isin(big_five_leagues)) &
    (F.col("season").isin(seasons)) &
    (F.col("total_games") > 10)
)

# Select and sort by performance in descending order
df_top_teams = df_top_teams.select(
    "season",
    "club_id",
    "club_name",
    F.col("domestic_competition_id").alias("dom_league"),
    "performance",
    "total_games",
    "total_market_value",
).orderBy(F.col("total_market_value").desc())

# Get the top 10 teams
df_top_teams_by_market_value_and_season = df_top_teams.limit(10)

df_top_teams_by_market_value_and_season.write.format("delta").mode("overwrite").saveAsTable("my_database.top_teams_by_market_value_and_season")
# Show the result
# df_top_teams_10.show()


In [0]:
# Importing required libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import pandas as pd

# Assuming df_top_teams is already a PySpark DataFrame, let's convert it to Pandas for plotting
df = df_top_teams.toPandas()  # Convert PySpark DataFrame to Pandas DataFrame

# Prepare the data
X = df[['total_market_value']].values  # Independent variable (market value)
y = df['performance'].values  # Dependent variable (performance)

# Create the linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict the values using the model
y_pred = model.predict(X)

# Calculate the R² value (coefficient of determination)
r2 = model.score(X, y)

# Plot the scatter plot and regression line
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['total_market_value'], y=df['performance'], color='blue', alpha=0.6, label='Data points')

# Plot the regression line
sns.lineplot(x=df['total_market_value'], y=y_pred, color='red', label=f"Trend line (R² = {r2:.2f})")

# Set plot titles and labels
plt.title(f"Market Value vs Performance (R² = {r2:.2f})")
plt.xlabel("Market Value diff (M€)")
plt.ylabel("Performance (%)")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [0]:
from pyspark.sql import functions as F


team_stats_with_transfers = spark.read.parquet("dbfs:/tmp/team_stats_with_transfers.parquet")
# Define the valid seasons
valid_seasons = list(range(2010, 2024))

# Filter data according to the given criteria
filtered_teams = team_stats_with_transfers.filter(
    (F.col('domestic_competition_id').isin(['L1', 'GB1', 'ES1', 'FR1', 'IT1'])) &
    (F.col('season').isin(valid_seasons)) &
    (F.col('total_games') > 10)
)

# # Select and sort by performance in descending order
# df_top_teams = df_top_teams.select(
#     "season",
#     "club_id",
#     "club_name",
#     F.col("domestic_competition_id").alias("dom_league"),
#     "performance",
#     "total_games",
#     "total_market_value",
# ).orderBy(F.col("total_market_value").desc())

# Group by club_id and calculate the necessary aggregations
top_market_value = filtered_teams.groupBy(
    'club_id', 'club_name', 'domestic_competition_id'
).agg(
    F.sum('total_market_value').alias('total_market_value'),
    # Calculate performance as specified
    F.round(
        (F.sum('win') * 3.0 + F.sum('draw')) * 100.0 / (F.sum('total_games') * 3.0), 2
    ).alias('performance'),
    F.sum('total_games').alias('total_games'),
    F.sum('win').alias('win'),
    F.sum('draw').alias('draw'),
    F.sum('lost').alias('lost'),
    F.sum('GF').alias('GF'),
    F.sum('GA').alias('GA')
)

# Calculate the goal difference (GD)
top_market_value = top_market_value.withColumn(
    'GD', F.col('GF') - F.col('GA')
)

# Sort by total_market_value and get the top 10 teams
df_top_teams_by_market_value_last_decade = top_market_value.select(
    "club_id",
    "club_name",
    F.col("domestic_competition_id").alias("dom_league"),
    "performance",
    "total_games",
    "total_market_value",
).orderBy(F.col('total_market_value'), ascending=False).limit(10)

df_top_teams_by_market_value_last_decade.write.format("delta").mode("overwrite").saveAsTable("my_database.top_teams_by_market_value_last_decade")
# Show the top 10 teams
# top_market_value.show(10)


In [0]:
# Importing required libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import pandas as pd

# Assuming df_top_teams is already a PySpark DataFrame, let's convert it to Pandas for plotting
df = top_market_value.toPandas()  # Convert PySpark DataFrame to Pandas DataFrame
df = df[df['total_market_value'] > 0].copy()
# Tạo mô hình hồi quy tuyến tính
X = df[['total_market_value']]
y = df['performance'].values  # Dependent variable (performance)

# Create the linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict the values using the model
y_pred = model.predict(X)

# Calculate the R² value (coefficient of determination)
r2 = model.score(X, y)

# Plot the scatter plot and regression line
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['total_market_value'], y=df['performance'], color='blue', alpha=0.6, label='Data points')

# Plot the regression line
sns.lineplot(x=df['total_market_value'], y=y_pred, color='red', label=f"Trend line (R² = {r2:.2f})")

# Set plot titles and labels
plt.title(f"Market Value vs Performance (R² = {r2:.2f})")
plt.xlabel("Market Value diff (M€)")
plt.ylabel("Performance (%)")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS my_database;

